## Repeated MCX simulation Stage 4 with multi-distance detector


In [ ]:
! pip install pmcx
! pip install numpy pmcx jdata bjdata matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for bjdata: filename=bjdata-0.4.1-cp311-cp311-linux_x86_64.whl size=22196 sha256=6bc6ef394c37fa426c732b1c189284f6df00bbd5659eec9605071d6a33078b8e
  Stored in directory: /root/.cache/pip/wheels/e4/d5/4a/8d5ccb70c0eb1f07a10da8548e50b7fc0cbdfd014f5eaa45c3
Successfully built bjdata


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
import os
import pmcx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pickle

Mounted at /content/drive


In [ ]:
# Path to your folder containing the .py script
folder_path = '/content/drive/MyDrive/Colab Notebooks/MCX_Sims/Simulation Data'

# Add to sys.path so you can import the script
if folder_path not in sys.path:
    sys.path.append(folder_path)
from FD_msNIRS import mcx_simulation as ms
from FD_msNIRS import mcx_fft as mf
from FD_msNIRS import extract_freq as ef

In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/MCX_Sims/Simulation Data/"

'10_mu_as_df 1.xlsx'
 10_mu_s_primes_df.xlsx
'30_mu_as_df 1.xlsx'
 30_mu_s_primes_df.xlsx
'Copy of 10_mu_as_df 1.xlsx'
'Copy of 10_mu_s_primes_df.xlsx'
 FD_msNIRS.py
 __pycache__
'Repeat_10_ MCX_simulation_multiDistance_bbNIRS_stage3_2.ipynb'


In [ ]:
pmcx.gpuinfo()

[{'name': 'NVIDIA A100-SXM4-40GB',
  'id': 1,
  'devcount': 1,
  'major': 8,
  'minor': 0,
  'globalmem': 42474471424,
  'constmem': 65536,
  'sharedmem': 49152,
  'regcount': 65536,
  'clock': 1410000,
  'sm': 108,
  'core': 6912,
  'autoblock': 64,
  'autothread': 442368,
  'maxgate': 0}]

### MCX Simulation using packgae

In [ ]:

# Load the uploaded Excel files
mu_as_path = "/content/drive/MyDrive/Colab Notebooks/MCX_Sims/Simulation Data/Copy of 10_mu_as_df 1.xlsx"
mu_s_primes_path = "/content/drive/MyDrive/Colab Notebooks/MCX_Sims/Simulation Data/Copy of 10_mu_s_primes_df.xlsx"

# Read the data from the files
mu_as_df = pd.read_excel(mu_as_path)
mu_s_primes_df = pd.read_excel(mu_s_primes_path)

# Display the first few rows of each to verify structure
mu_as_df.head(), mu_s_primes_df.head()


(        784       800       818       835       851       868       881  \
 0  0.216999  0.221568  0.236370  0.260919  0.274617  0.284475  0.290162   
 1  0.250405  0.256018  0.273232  0.300210  0.316056  0.327906  0.334864   
 
         894  
 0  0.294153  
 1  0.339831  ,
          784        800        818        835        851        868       881  \
 0  11.697548  11.210975  11.098415  10.437974  10.499046   9.880689  9.785651   
 1  11.910163  11.373240  10.942023  10.624372  10.172231  10.137458  9.762257   
 
         894  
 0  9.561698  
 1  9.416336  )

In [ ]:


g=0.85
wls = [784,800,818,835,851,868,881,894]
distance = [2, 2.5, 3.0 ,3.5] # cmimport ioimport io
distance_mm = [d * 10 for d in distance]  # mm


# Iterate over each row (i.e., simulation)
for sim_idx, (ua_row, us_p_row) in enumerate(zip(mu_as_df.values, mu_s_primes_df.values)):
    ua = np.array(ua_row)
    us_prime = np.array(us_p_row)
    us = us_prime / (1 - g)

    ua_mm = ua / 10
    us_mm = us / 10
    print('ua = ', ua_mm)
    print('us = ', us_mm)
    print(f"\n--- Simulation {sim_idx + 1} ---")

    # -------- Simulation for Multi-Distances --------------
    dictt_ms = {}
    for j in range(ua_mm.shape[0]):
        print('ua = ', ua_mm[j])
        print('us = ', us_mm[j])
        TPSF_list, time_unit = ms(ua_mm[j], us_mm[j], g=0.85, n=1.370, distance=distance_mm, tend=1e-08, devf=1000, nphoton=1.2e8)

        for i, d in enumerate(distance_mm):
            label = f"{d}, {wls[j]}"
            dictt_ms[label] = [TPSF_list[i], time_unit]

    # -------- Frequency domain analysis --------------
    target_freq = 110e6
    uac = []
    udc = []
    phase1 = []
    phase2 = []
    labels = []

    for label, (TPSF, tstep) in dictt_ms.items():
        uac_i, udc_i, p1, p2 = ef(target_freq, [TPSF], tend=1e-08, devf=1000)
        uac.append(uac_i[0])
        udc.append(udc_i[0])
        phase1.append(p1[0])
        phase2.append(p2[0])
        labels.append(label)
    # Group data by wavelength
    data_by_wl = {}

    # Reconstruct the distances and wavelengths from the labels
    for (key, (TPSF, tstep)), uac_i, udc_i, p1, p2 in zip(dictt_ms.items(), uac, udc, phase1, phase2):
        dist_str, wl_str = key.split(',')  # split key: "distance, wavelength"
        wl = float(wl_str.strip())
        d = float(dist_str.strip())

        if wl not in data_by_wl:
            data_by_wl[wl] = {'distance': [], 'uac': [], 'udc': [], 'phase': []}

        data_by_wl[wl]['distance'].append(d)
        data_by_wl[wl]['uac'].append(uac_i)
        data_by_wl[wl]['udc'].append(udc_i)
        data_by_wl[wl]['phase'].append(p1)

    # Sort the wavelengths for consistent plotting
    sorted_wls = sorted(data_by_wl.keys())


    # -------- Save Data --------------

    columns = []
    row = []

    # Get sorted lists to ensure consistent ordering
    sorted_wls = sorted(data_by_wl.keys())
    sorted_distances = sorted({d for data in data_by_wl.values() for d in data['distance']})

    for i, distance in enumerate(sorted_distances):  # i = distance index
        for j, wl in enumerate(sorted_wls):         # j = wavelength index
            data = data_by_wl[wl]
            if distance in data['distance']:
                idx = data['distance'].index(distance)

                # Use index-based naming like U_AC_1_1
                columns.extend([
                    f"U_AC_{i+1}_{j+1}",
                    f"U_DC_{i+1}_{j+1}",
                    f"Phase_{i+1}_{j+1}"
                ])

                row.extend([
                    data['uac'][idx],
                    data['udc'][idx],
                    data['phase'][idx]
                ])

    df = pd.DataFrame([row], columns=columns)
    sim_filename = f"simulation_{sim_idx+6:02d}.csv"
    df.to_csv(sim_filename, index=False)
    print('-------------------------    Saved Data Successfully!!!! ----------------------------')

ua =  [0.02169992 0.02215682 0.02363704 0.02609189 0.02746173 0.02844752
 0.02901616 0.02941533]
us =  [7.79836522 7.47398338 7.39894336 6.95864912 6.9993643  6.5871262
 6.52376701 6.37446524]

--- Simulation 1 ---
ua =  0.0216999183813259
us =  7.798365223254265
nphoton: 1.2e+08
tstart: 0
tstep: 1e-11
tend: 1e-08
maxdetphoton: 1.2e+08
issrcfrom0: 1
autopilot: 1
unitinmm: 1
issaveref: 1
issavedet: 1
ua =  0.0221568215472919
us =  7.473983375361199
nphoton: 1.2e+08
tstart: 0
tstep: 1e-11
tend: 1e-08
maxdetphoton: 1.2e+08
issrcfrom0: 1
autopilot: 1
unitinmm: 1
issaveref: 1
issavedet: 1
ua =  0.0236370354938161
us =  7.398943364716466
nphoton: 1.2e+08
tstart: 0
tstep: 1e-11
tend: 1e-08
maxdetphoton: 1.2e+08
issrcfrom0: 1
autopilot: 1
unitinmm: 1
issaveref: 1
issavedet: 1
ua =  0.026091894245315002
us =  6.958649116855932
nphoton: 1.2e+08
tstart: 0
tstep: 1e-11
tend: 1e-08
maxdetphoton: 1.2e+08
issrcfrom0: 1
autopilot: 1
unitinmm: 1
issaveref: 1
issavedet: 1
ua =  0.027461726153565402
us =